In [2]:
"""
    problem tanımı: Öneri sistemleri
    user(kullanıcı) - item(ürünler) - rating(puanlar)
"""

#import libraries
import numpy as np

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Embedding,Flatten,Dot,Dense
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

In [4]:
#veri setini oluşturalım
user_ids = np.array([0,1,2,3,4,0,1,2,3,4])
items_id = np.array([0,1,2,3,4,1,2,3,4,5])
ratings = np.array([5,4,3,2,1,4,3,2,1,5])

#train -test split
user_ids_train,user_ids_test, items_id_train,_items_id_test,ratings_train,ratings_test = train_test_split(user_ids,items_id,ratings,test_size=0.2,random_state=42)

In [18]:
#create neural network
def create_model(num_users,num_items,embedding_dim):

     # Kullanıcı ve öğe ID giriş tensorleri
    user_input = Input(shape=(1,), name="user_input")
    item_input = Input(shape=(1,), name="item_input")
    
    # Embedding katmanları
    user_embedding = Embedding(input_dim=num_users, output_dim=embedding_dim, name="user_embedding")(user_input)
    item_embedding = Embedding(input_dim=num_items, output_dim=embedding_dim, name="item_embedding")(item_input)
    
    # Vektörleri düzleştir
    user_vecs = Flatten()(user_embedding)
    item_vecs = Flatten()(item_embedding)

     # Nokta çarpımı
    dot_product = Dot(axes=1)([user_vecs, item_vecs])
    
    # Model oluştur
    model = Model(inputs=[user_input, item_input], outputs=dot_product)
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    return model


In [17]:
#train test
num_users=5
num_items=6
embedding_dim = 8
model= create_model(num_users,num_items,embedding_dim)
model.fit([user_ids_train,items_id_train],ratings_train,epochs=10,verbose=1, validation_split =0.1)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 12.0002 - val_loss: 8.9981
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - loss: 11.9983 - val_loss: 8.9977
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - loss: 11.9964 - val_loss: 8.9973
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - loss: 11.9946 - val_loss: 8.9969
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - loss: 11.9927 - val_loss: 8.9964
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - loss: 11.9908 - val_loss: 8.9960
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 11.9889 - val_loss: 8.9956
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - loss: 11.9870 - val_loss: 8.9951
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - loss: 11.9850 - val_loss: 8.9947
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - loss: 11.9831 - val_loss: 8.9942


In [19]:
loss = model.evaluate([user_ids_test,_items_id_test],ratings_test)
loss

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 8.5005


8.50054931640625

In [22]:
user_id = np.array([0])
item_id = np.array([5])
prediction = model.predict([user_id,item_id])

print(f"Predicted rating for user: {user_id[0]}, item:  {item_id[0]},prediction: {prediction[0][0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predicted rating for user: 0, item:  5,prediction: -0.0034220051020383835


MAKİNE ÖĞRENİMİ

In [ ]:
from surprise import Dataset, KNNBasic,accuracy
from surprise.model_selection import train_test_split

#import versi seti
data = Dataset.load_builtin("ml-100k") #kullanıcı_id,film_id, puan

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\kezer/.surprise_data/ml-100k


In [27]:
#train test split
trainset, testset= train_test_split(data,test_size=0.2)

In [30]:
#Ml model create: KNN
model_options = {
    "name":"cosine",
    "user_based":True #kullanıcı arası benzerlik
}

#train
model = KNNBasic(sim_options=model_options)
model.fit(trainset)

#test
prediction = model.test(testset)
accuracy.rmse(prediction)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0180


1.0179867437324714

In [31]:
#recommendation systme
def get_top_n(predictions, n=10):

    top_n = {}

    for uid, iid, true_r, est, _ in predictions:
        if not top_n.get(uid):
            top_n[uid] =[]
        top_n[uid].append((iid,est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key = lambda x: x[1],reverse= True)
        top_n[uid] = user_ratings[:n]
    
    return top_n


In [32]:
n=5
top_n = get_top_n(prediction,n)

user_id= "2"

print(f"top {n} recommendation for user {user_id}")
for item_id,rating in top_n[user_id]:
    print(f"item: {item_id}, score:{rating}")

top 5 recommendation for user 2
item: 272, score:4.299394180633609
item: 50, score:4.225312863950581
item: 258, score:4.02501435006938
item: 10, score:3.9271081909206744
item: 283, score:3.8748093941465567
